In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
spark = SparkSession.builder.appName('Graphs-HW2').getOrCreate()

In [2]:
from pyspark.sql.functions import lit

In [3]:
import pyspark.sql.functions as sf

In [4]:
# Read lines from the text file
pr_sdf = spark.read.load('pr_graph.txt', format="text")

In [5]:
pr_sdf.createOrReplaceTempView('pr_sdf_view')
pr_sdf = spark.sql("SELECT CAST(split(value, ' ')[0] AS int) AS from_node, CAST(split(value, ' ')[1] AS int) AS to_node FROM pr_sdf_view")

In [6]:
pr_sdf.show(5)

+---------+-------+
|from_node|to_node|
+---------+-------+
|        1|      2|
|        1|      3|
|        1|      4|
|        1|      5|
|        2|      3|
+---------+-------+
only showing top 5 rows



In [7]:
# from_node, weight
def helpera(G):
    outdegree_sdf = G.groupBy("from_node").count()
    outdegree_sdf = outdegree_sdf.selectExpr("from_node","count as outdegree")
    my_fn = F.udf(lambda x: 1/x, DoubleType())
    outdegree_sdf = outdegree_sdf.select('from_node',my_fn(outdegree_sdf['outdegree']).alias('weight'))
    outdegree_sdf.cache()
    return outdegree_sdf

In [9]:
#from_node, to_node, weight
def helperb(G, out):
    return_sdf = G.alias('g').join(out.alias('o'),'from_node','left')
    return return_sdf

In [11]:
def pagerank(G, num_iter):
    #Your logic goes here
    all_node = G.select(G['from_node'].alias('node')).unionAll(G.select(G['to_node'].alias('node'))).distinct()
    #calculate initial pagerank
    init_pr = 1/all_node.count()
    helper1 = helpera(pr_sdf)
    helper2 = helperb(pr_sdf,helper1)
    helper2 = helper2.withColumn('from_pr',lit(init_pr))
    my_fn = F.udf(lambda x, y: x*y, DoubleType())
    #calculate to_pr --> cur_pr
    helper3 = helper2.select('from_node','to_node','weight','from_pr',my_fn('weight','from_pr').alias('avg_pr'))
    helper3 = helper3.groupBy("to_node").agg(sf.sum('avg_pr').alias('tot_pr'))
    cal_fn = F.udf(lambda x: 0.85*x+0.15, DoubleType())
    helper3 = helper3.select('to_node',cal_fn('tot_pr').alias('to_pr'))
    helper2 = helper2.join(helper3,'to_node','left')
    helper1.cache()
    helper2.cache()
    helper3.cache()
    #iterate
    i=1
    while i<num_iter:
        helper2 = helperb(pr_sdf,helper1)
        #update helper2 by changing from_pr to cur_pr
        helper3 = helper3.selectExpr("to_node as from_node","to_pr as from_pr")
        helper2 = helper2.alias('a').join(helper3.alias('b'),'from_node','left')
        my_fn = F.udf(lambda x, y: x*y, DoubleType())
        helper3 = helper2.select('from_node','to_node','weight','from_pr',my_fn('weight','from_pr').alias('avg_pr'))
        helper3 = helper3.groupBy("to_node").agg(sf.sum('avg_pr').alias('tot_pr'))
        cal_fn = F.udf(lambda x: 0.85*x+0.15, DoubleType())
        helper3 = helper3.select('to_node',cal_fn('tot_pr').alias('to_pr'))
        helper2 = helper2.join(helper3,'to_node','left')
        helper1.cache()
        helper2.cache()
        helper3.cache()
        i=i+1
    pr_values_sdf = helper2.selectExpr('to_node as node','to_pr as pagerank').distinct()
    return pr_values_sdf

## Step 5

In [12]:
pr_sdf.count()

16

In [13]:
pr_sdf.show()

+---------+-------+
|from_node|to_node|
+---------+-------+
|        1|      2|
|        1|      3|
|        1|      4|
|        1|      5|
|        2|      3|
|        2|      5|
|        3|      2|
|        4|      5|
|        5|      1|
|        5|      6|
|        5|      7|
|        6|      7|
|        7|      6|
|        7|      2|
|        7|      7|
|        5|      4|
+---------+-------+



In [14]:
pagerank(pr_sdf, 5).orderBy("node").show()

+----+-------------------+
|node|           pagerank|
+----+-------------------+
|   1| 0.3056855115440409|
|   2|  0.843575161873236|
|   3| 0.5262470701160463|
|   4|0.36610546184882914|
|   5| 0.8147069488207835|
|   6| 0.5380753784802238|
|   7| 0.9433725923168398|
+----+-------------------+

